<a href="https://colab.research.google.com/github/SebasRaul2503/Pet-Recognition/blob/main/Despliegue_de_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Clonamos repo de los modelos
!git clone https://github.com/SebasRaul2503/Pet-Recognition.git

Cloning into 'Pet-Recognition'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 31 (delta 3), reused 17 (delta 2), pack-reused 0
Receiving objects: 100% (31/31), 7.85 KiB | 7.85 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Filtering content: 100% (6/6), 442.49 MiB | 45.91 MiB/s, done.


In [ ]:
#@title Instalamos dependencias necesarias
!pip install flask-ngrok

In [ ]:
#@title Importacion de las bibliotecas necesarias
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import numpy as np
from PIL import Image
import io
import os
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from werkzeug.utils import secure_filename

In [ ]:
#@title Carga de los 6 modelos
model_1 = tf.keras.models.load_model('/content/Pet-Recognition/Models/InceptionV3_09210.keras')
model_2 = tf.keras.models.load_model('/content/Pet-Recognition/Models/InceptionV3_09362_noLSTM.keras')
model_3 = tf.keras.models.load_model('/content/Pet-Recognition/Models/VGG16_08852.keras')
model_4 = tf.keras.models.load_model('/content/Pet-Recognition/Models/VGG16_08898_noLSTM.keras')
model_5 = tf.keras.models.load_model('/content/Pet-Recognition/Models/noPre_06120.keras')
model_6 = tf.keras.models.load_model('/content/Pet-Recognition/Models/noPre_08942_noLSTM.keras')

# Define una lista con los nombres de los modelos correspondientes
model_names = [
    'InceptionV3_09210',
    'InceptionV3_09362_noLSTM',
    'VGG16_08852',
    'VGG16_08898_noLSTM',
    'noPre_06120',
    'noPre_08942_noLSTM'
]

In [ ]:
#@title Vemos la URL que usaremos para la interfaz (Siempre que se reinicie la sesion del Colab, tenemos que correr esta celda, porque cambia)
from google.colab.output import eval_js
url = eval_js("google.colab.kernel.proxyPort(5000)")
print(url)

https://ww3fvewtsan-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
#@title Usamos Flask para desplegar los modelos en la web
def predict_image(image_path, model):
    img = image.load_img(image_path, target_size=(128, 128))  # Ajusta el tamaño segun el modelo
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)  # Añade una dimension para el lote
    img_array /= 255.0  # Normaliza los valores de pixeles al rango [0, 1]
    prediction = model.predict(img_array)
    return "Perro" if prediction[0][0] > 0.5 else "Gato"

# Configuración de la aplicación Flask
app = Flask(__name__, template_folder='/content/Pet-Recognition/templates')  # Corrige la ruta del directorio de templates
run_with_ngrok(app)  # Iniciar ngrok cuando la aplicación se ejecute

# Directorio para almacenar los archivos temporales
UPLOAD_FOLDER = '/content/uploads/'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ruta principal de la aplicación
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        if file:
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)  # Guardar el archivo en el directorio de uploads

            predictions = {}  # Diccionario para almacenar las predicciones
            for i in range(1, 7):
              model_name = model_names[i-1]
              prediction = predict_image(file_path, globals()[f'model_{i}'])  # Obtiene el modelo correspondiente por su nombre
              predictions[model_name] = prediction

            os.remove(file_path)  # Eliminar el archivo temporal después de las predicciones
            return predictions
    return render_template('index.html', prediction=None)

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run()